In [2]:
# read in the text, transforming everything to lower case
text = open('Beiras.txt').read().lower()
print('our original text has ' + str(len(text)) + ' characters')


our original text has 589161 characters


In [4]:
chars=sorted(list(set(text)))
print(chars);

['\n', ' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'ª', '«', '´', 'º', '»', '¿', 'à', 'á', 'â', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü', '–', '—', '‘', '’', '“', '”', '•', '…']


In [11]:
import re
cleanString = re.sub('\W+',' ',text)

chars=sorted(list(set(cleanString )))
print(chars);

[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ª', 'º', 'à', 'á', 'â', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü']


In [7]:
print(cleanString[1000:])

a as suas cultas señorías de que é a reprodución literal dun punxente poema de rosalía titulado a xusticia pola man e dado á luz no seu libro follas novas por certo que se rosalía publicase arestora ese mesmo poema non sería alcuñada de tola senón se cadra imputada por apoloxía do terrorismo ou acusada de incitación á sublevación popular e á desorde pública que todo pode ser nesta especie de franquismo sen franco que estamos a disfroitar dirán vostedes que iso todo é pasado e non ven a conto hoxe pois co seu permiso discrépolles para min o equivalente actual da muller protagonista anónima do poema rosalián alén de calquera das mulleres maltratadas hoxendía tamén leva nome femenino élles galiza élles a cidadanía galega do común roubada espoliada asoballada desprezada aldraxada vítima da inmisericorde cobiza da plutocracia financeira e da inxustiza empoleirada no poder político unha cidadanía inerme acosada a tódolos níveis e en tódalas dimensións dos seus dereitos liberdades e condición